In [ ]:
import ee
import geemap
import pandas as pd
import gee.gee_objects as gee
import datetime as dt
import math
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
def equation_of_time_minutes(utc_dt: dt.datetime) -> float:
    """
    NOAA formulation (minutes). Works best with UTC datetime (timezone-aware or naive as UTC).
    """
    # Ensure we use UTC date/time
    if utc_dt.tzinfo is not None:
        utc_dt = utc_dt.astimezone(dt.timezone.utc).replace(tzinfo=None)

    year_start = dt.datetime(utc_dt.year, 1, 1)
    day_of_year = (utc_dt - year_start).days + 1
    # fractional hour in UTC
    frac_hour = utc_dt.hour + utc_dt.minute/60 + utc_dt.second/3600
    # fractional year (radians); NOAA variant
    gamma = 2*math.pi/365 * (day_of_year - 1 + (frac_hour - 12)/24)

    eot = (229.18 * (0.000075
                     + 0.001868*math.cos(gamma)
                     - 0.032077*math.sin(gamma)
                     - 0.014615*math.cos(2*gamma)
                     - 0.040849*math.sin(2*gamma)))
    return eot  # minutes (can be negative)


def utc_to_local_solar_time(utc_millis: int, longitude_deg: float):
    utc =  dt.datetime.fromtimestamp(utc_millis/1000, tz=dt.timezone.utc)
    lon_offset_min = 4.0 * longitude_deg
    lmst = utc + dt.timedelta(minutes=lon_offset_min)

    eot_min = equation_of_time_minutes(utc)
    ast = lmst + dt.timedelta(minutes=eot_min)
    return ast

In [ ]:
vector_BLS = ee.FeatureCollection('projects/saw-ucdavis/assets/BLS')
vector_RIP720 = ee.FeatureCollection('projects/saw-ucdavis/assets/RIP_720')

vector = vector_BLS
farm = 'BLS'
first_date = '2024-01-01'
last_date = '2025-12-31'
crs = 'EPSG:32610'
centroid = vector.geometry().centroid().getInfo()['coordinates']
lon = centroid[0]
lat = centroid[1]

In [ ]:
S2_BLS = (gee.Sentinel2(farm=farm, aoi=vector)
          .filter_date(first_date, last_date)
          .cloud_mask())

S2_BLS = S2_BLS.clip()
S2_BLS.percentage_pixel_free_clouds(band='B4', scale=10, crs=crs)
S2_BLS.filter_by_feature(filter='gte', name='percentage_pixel_free_clouds', value=99)

In [ ]:
# Getting water vapor pressure and aerosol optical thickness

In [151]:
def reduceRegion(image, band, vector, reducer='mean', scale=10, crs='EPSG:32610'):
    reducer = (image.select(band)
               .reduceRegion(reducer=reducer, geometry=vector, scale=scale, crs=crs)
               .set('system:time_start', image.get('system:time_start'))
               .set('MEAN_SOLAR_AZIMUTH_ANGLE', image.get('MEAN_SOLAR_AZIMUTH_ANGLE'))
               .set('MEAN_SOLAR_ZENITH_ANGLE', image.get('MEAN_SOLAR_ZENITH_ANGLE'))
               .set('MEAN_INCIDENCE_ZENITH_ANGLE_B8', image.get('MEAN_INCIDENCE_ZENITH_ANGLE_B8'))
               )
    reducer = reducer.getInfo()
    return reducer

dates = S2_BLS.get_feature('date', unique=True)
data_stats = [reduceRegion(S2_BLS.get_image(i), band=['AOT', 'WVP'], vector=vector, scale=20, crs=crs)
                      for i in dates]
data_stats = pd.DataFrame(data_stats)
data_stats['overpass_solar_time'] = data_stats['system:time_start'].map(
    lambda x: utc_to_local_solar_time(x, longitude_deg=lon)
)
data_stats = data_stats[['overpass_solar_time', 'MEAN_SOLAR_AZIMUTH_ANGLE', 'MEAN_SOLAR_ZENITH_ANGLE', 'MEAN_INCIDENCE_ZENITH_ANGLE_B8', 'AOT', 'WVP']]
data_stats.WVP = data_stats.WVP * 0.001
data_stats.AOT = data_stats.AOT * 0.001
data_stats.to_csv(rf'C:\Users\mqalborn\Desktop\ET_3SEB\GRAPEX\METADATA.csv', index=False)

In [ ]:
out_dir = rf'C:\Users\mqalborn\Desktop\ET_3SEB\GRAPEX\Sentinel2'
geemap.ee_export_image_collection(
    S2_BLS.gee_image_collection.select('B.*'),
    scale=10,
    region=S2_BLS.aoi.geometry(),
    out_dir=out_dir)

In [ ]:
Map = geemap.Map()
Map.centerObject(vector, zoom=16)

vis_params = {
    'bands': ['B4', 'B3', 'B2'], # False color composite (NIR, Red, Green)
    'min': 0.0,
    'max': 3000,
}

Map.add_layer(S2image, vis_params, "Sentinel2")
# Map.add_layer(vector, {'color': 'FF0000', 'opacity': 0.5}, "BLS")

Map

In [ ]:
data = S2_BLS.count_pixels_features(index='B4', scale=10, crs='EPSG:32610')

In [ ]:
filename = rf'C:\Users\mqalborn\Desktop\ET_3SEB\raster/example.tif'
geemap.ee_export_image(S2image, filename=filename, scale=10, region=S2_BLS.aoi.geometry(),  file_per_band=False)

In [ ]:
out_dir = rf'C:\Users\mqalborn\Desktop\ET_3SEB\PISTACHIO\Sentinel2'
geemap.ee_export_image_collection(
    S2_BLS.gee_image_collection.select('B.*'),
    scale=10,
    region=S2_BLS.aoi.geometry(),
    out_dir=out_dir)

In [ ]:
Map = geemap.Map()
Map.centerObject(vector, zoom=16)

vis_params = {
    'bands': ['B4', 'B3', 'B2'], # False color composite (NIR, Red, Green)
    'min': 0.0,
    'max': 3000,
}

Map.add_layer(S2image, vis_params, "Sentinel2")
# Map.add_layer(vector, {'color': 'FF0000', 'opacity': 0.5}, "BLS")
Map


In [ ]:
start_date = '2024-01-01'
end_date = '2025-01-01'
vector = vector_BLS
S2_BLS = get_s2_sr_cld_col(vector, start_date=start_date, end_date=end_date)
S2_BLS_median = (S2_BLS.map(add_cld_shdw_mask)
                             .map(apply_cld_shdw_mask)
                             .median())

In [ ]:
visParams = {
  'min': 0.0,
  'max': 3000.0,
  'bands': ['B4', 'B3', 'B2'],
}

def visualization(img):
    return img.visualize(visParams).clip(vector)
# Create RGB visualization images for use as animation frames.
rgbVis = S2_BLS.map(visualization)

In [ ]:
gifParams = {
  'region': vector.geometry(),
  'dimensions': 600,
  # 'crs': 'EPSG:3857',
  'framesPerSecond': 10
}

print(rgbVis.getVideoThumbURL(gifParams))